In [1]:
from environment import Environment
from models.baseline import BaselineModel
import pandas as pd

In [2]:
data = pd.read_csv('data/train_preprocessed.csv')
test_data = pd.read_csv('data/validate_preprocessed.csv')

In [3]:
env = Environment(data)

In [4]:
model = BaselineModel(test_data)

In [ ]:
while True:
    action = env.action_space.sample()  # Random action selection
    obs, reward, done, _ = env.step(action)
    env.render()
    print('Reward:', reward)
    print('Done:', done)